In [1]:
from source.regency import Regency
from source.run import rungame
from source.mapping import Mapping
from source.DQNAgent import DQNAgent
from IPython.display import clear_output
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
%matplotlib notebook

In [3]:
X = 5
my_game = rungame('TestGame', train=True, GameLength=X, IntDC = 15)
Game = my_game.Game


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Continuing Game TestGame at Season 3 Action 3


In [13]:
Game.random_events()
Game.collect_regency_points()
Game.domain_initiative()
Game.collect_gold_bars()
Game.maintenance_costs(Update=True)
Game.set_override('HK',45)
Game.take_domain_actions()
Game.take_domain_actions()
Game.take_domain_actions()
Game.War

Time to Finish the Rounditiative  1
Time to Finish the Rounditiative  1
Time to Finish the Rounditiative  1
   Regent                                          Full Name  Final Score  First Score  Score
40     EM    Duchess Eriene Mierelen (Brosengae Royal Guild)          398           50    348
69     AB                          Aeric Boeruine (Boeruine)          386          188    198
12     CA                                       Carilon Alam          220           65    155
56    WIT  Western Imperial Temple of Haelyn (Rhobber Nic...          202           75    127
64     Is                              Emerald Queen Isaelie          207          104    103
45     DW       Diirk Watersold (Royal Guild of Baruk-Azhir)           96           26     70
87     AI                                        Arlen Innis           94           25     69
50    GTr                   Kral Two-Toes (Goblin's Triumph)           79           17     62
52     MR                                     

,Year,Location,Event,Notes
0,"Taelinir, year 1524 HC",Conallier,Occupation of Conallier,"Marlae Roesone's forces occupied Conallier, qu..."
0,"Roelir, year 1524 HC",Conallier,Occupation of Conallier,"Marlae Roesone's forces occupied Conallier, sa..."
0,"Haelynir, year 1524 HC",Conallier,Occupation of Conallier,"Marlae Roesone's forces occupied Conallier, sa..."
0,"Anarire, year 1524 HC",Conallier,Occupation of Conallier,"Marlae Roesone's forces occupied Conallier, sa..."
0,"Deismir, year 1524 HC",Conallier,Occupation of Conallier,"Marlae Roesone's forces occupied Conallier, sa..."
0,"Emmanir, year 1524 HC",Marilen,Occupation of Marilen,"Harald Khorien's forces occupied Marilen, quas..."
0,"Faniele, year 1524 HC",Seasedge,Battle of Seasedge,Battle of Seasedge\n\nBelligerents:\nArien Bor...
0,"Faniele, year 1524 HC",Tariene,Battle of Tariene,Battle of Tariene\n\nBelligerents:\nArien Bort...
0,"Sarimiere, year 1525 HC",Anuire,Battle of Anuire,Battle of Anuire\n\nBelligerents:\nHarald Khor...
0,"Taelinir, year 1525 HC",Pechalinn,Occupation of Pechalinn,"Rhuobhe ""Manslayer""'s forces occupied Pechalin..."


In [11]:
self.War

,Year,Location,Event,Notes
0,"Taelinir, year 1524 HC",Conallier,Occupation of Conallier,"Marlae Roesone's forces occupied Conallier, qu..."
0,"Roelir, year 1524 HC",Conallier,Occupation of Conallier,"Marlae Roesone's forces occupied Conallier, sa..."
0,"Haelynir, year 1524 HC",Conallier,Occupation of Conallier,"Marlae Roesone's forces occupied Conallier, sa..."
0,"Anarire, year 1524 HC",Conallier,Occupation of Conallier,"Marlae Roesone's forces occupied Conallier, sa..."
0,"Deismir, year 1524 HC",Conallier,Occupation of Conallier,"Marlae Roesone's forces occupied Conallier, sa..."
0,"Emmanir, year 1524 HC",Marilen,Occupation of Marilen,"Harald Khorien's forces occupied Marilen, quas..."
0,"Faniele, year 1524 HC",Seasedge,Battle of Seasedge,Battle of Seasedge\n\nBelligerents:\nArien Bor...
0,"Faniele, year 1524 HC",Tariene,Battle of Tariene,Battle of Tariene\n\nBelligerents:\nArien Bort...
0,"Sarimiere, year 1525 HC",Anuire,Battle of Anuire,Battle of Anuire\n\nBelligerents:\nHarald Khor...


In [12]:
print(Game.War['Notes'].values[-1])

Battle of Anuire

Belligerents:
Harald Khorien [A] vs. Duchess Eriene Mierelen (Brosengae Royal Guild) [D]

Strength:
[Harald Khorien]

 Darksjoy  Coaster

[Duchess Eriene Mierelen (Brosengae Royal Guild)]
 Cavalry  1
  The Sun's Vengence  Galleon
 The Troubador's Eye  Galleon
   Haelyn's Begining  Galleon
      The Seas' Wing  Galleon
 The Oceans' Victory  Galleon
   Lady Halie's Rose  Galleon
           Riverdusk  Galleon
        The Red Gale  Caravel
    The Ranger's Eye  Caravel
 The Wizard's Bounty  Caravel
 The Faerie's Runner  Caravel
     Haelyn's Marlin  Caravel
       Avanil's Gale  Coaster
 The Lady's Begining  Coaster
      Erntenir's Gem  Coaster

Casualties:
[Harald Khorien]

 Darksjoy  Coaster
[Duchess Eriene Mierelen (Brosengae Royal Guild)]

 Erntenir's Gem  Coaster

Result: Duchess Eriene Mierelen (Brosengae Royal Guild) victory after 1 hours of fighting.


In [7]:
import numpy as np
import pandas as pd
self = Game

Regent, decision, actor, Type, state, capital, high_pop, low_pop, friend, enemy, rando, enemy_capital = Game.errors[0]

IndexError: list index out of range

In [ ]:
Game.Troops[Game.Troops['Provence'].str.contains('Con')]

In [ ]:
self.Provences[self.Provences['Provence'].str.contains('Con')]